<h1 align="center"> Лабораторна робота №3 </h1>
<h2 align="center"> <b> Криптоаналіз афінної біграмної підстановки </b> </h2>
<hr> <h3 align="center">Виконав студент групи ФБ-35 Куб'юк Дмитро </h3>

Мета роботи <br>
Набуття навичок частотного аналізу на прикладі розкриття моноалфавітної підстановки; опанування прийомами роботи в модулярній арифметиці.

In [1]:
import re
import math
from itertools import permutations

test_uf8 = "data\\13.txt"
with open(test_uf8, 'r', encoding='utf-8') as file:
    text = file.read().lower()

text = re.sub(r'[^а-яё]', '', text)

symbols = "абвгдежзийклмнопрстуфхцчшщьыэюя"
m = len(symbols)
mod = m*m

In [2]:
def bigram_to_int(bg):
    return symbols.index(bg[0]) * m + symbols.index(bg[1])

def int_to_bigram(x):
    return symbols[x // m] + symbols[x % m]

Реалізувати підпрограми із необхідними математичними операціями:

Обчисленням оберненого елементу за модулем із використанням розширеного алгоритму Евкліда.

In [3]:
def egcd(a, b):
    if b == 0:
        return a, 1, 0
    g, x1, y1 = egcd(b, a % b)
    return g, y1, x1 - (a // b) * y1

def modinv(a, n):
    g, x, y = egcd(a, n)
    if g != 1:
        return None
    return x % n

Розв’язуванням лінійних порівнянь. При розв’язуванні порівнянь потрібно коректно обробляти випадок із декількома розв’язками, повертаючи їх усі.

In [4]:
def solve_linear(a, b, n):
    g = math.gcd(a, n)
    if b % g != 0:
        return []

    a1 = a // g
    b1 = b // g
    n1 = n // g

    inv = modinv(a1, n1)
    if inv is None:
        return []

    x0 = (inv * b1) % n1
    return [(x0 + k * n1) % n for k in range(g)]

За допомогою програми обчислення частот біграм, яка написана в ході виконання комп’ютерного практикуму №1, знайти 5 найчастіших біграм запропонованого шифртексту (за варіантом). 

In [5]:
def get_freqs(text):
    freqs = {}
    for i in range(0, len(text) - 1, 2):
        bi = text[i:i+2]
        freqs[bi] = freqs.get(bi, 0) + 1

    sorted_freqs = dict(sorted(freqs.items(), key=lambda item: item[1], reverse=True))
    return sorted_freqs

bi_freqs = get_freqs(text)
print(bi_freqs)

{'аф': 53, 'яф': 53, 'дю': 43, 'ап': 41, 'нф': 41, 'хб': 41, 'ло': 40, 'яю': 38, 'ын': 37, 'рз': 37, 'кэ': 36, 'як': 35, 'вэ': 34, 'жн': 33, 'хэ': 33, 'ыф': 33, 'мф': 32, 'эв': 32, 'фэ': 31, 'вф': 30, 'ню': 30, 'го': 30, 'жб': 30, 'мю': 28, 'сю': 28, 'жв': 28, 'вы': 27, 'бэ': 26, 'яп': 26, 'йи': 26, 'шз': 26, 'ун': 25, 'рю': 25, 'мп': 24, 'ец': 24, 'лз': 24, 'тф': 23, 'ья': 22, 'хх': 22, 'эн': 22, 'ьэ': 21, 'уб': 21, 'аи': 21, 'ле': 21, 'юо': 19, 'щю': 19, 'нб': 19, 'лы': 19, 'эо': 18, 'тк': 18, 'ев': 18, 'пй': 18, 'нч': 18, 'щк': 17, 'ац': 17, 'ею': 17, 'он': 17, 'чв': 17, 'юэ': 16, 'щп': 16, 'уф': 16, 'щф': 16, 'лп': 16, 'ьи': 16, 'иф': 15, 'йь': 15, 'ип': 15, 'жп': 15, 'нц': 14, 'ео': 14, 'ик': 14, 'ру': 14, 'тн': 13, 'шэ': 13, 'шы': 13, 'щз': 13, 'кй': 13, 'вр': 13, 'зк': 13, 'мо': 13, 'дч': 12, 'ая': 12, 'сь': 12, 'цн': 12, 'ии': 12, 'пв': 12, 'оц': 12, 'ыд': 12, 'иэ': 12, 'оэ': 11, 'цб': 11, 'лю': 11, 'шу': 11, 'ях': 11, 'жх': 11, 'ыэ': 11, 'фп': 11, 'ке': 11, 'ук': 11, 'уы': 11,

Перебрати можливі варіанти співставлення частих біграм мови та частих біграм шифртексту (розглядаючи пари біграм із п’яти найчастіших). Для кожного співставлення знайти можливі кандидати на ключ (a, b), шляхом розв’язання системи (1).

In [6]:
def decrypt(ciphertext, a, b):
    ainv = modinv(a, m*m)
    if ainv is None:
        return None

    result = []
    for i in range(0, len(ciphertext) - 1, 2):
        Y = bigram_to_int(ciphertext[i:i+2])
        X = (ainv * (Y - b)) % (mod)
        result.append(int_to_bigram(X))
    return "".join(result)

In [7]:
def entropy(text):
    p = {}

    for char in text:
        p[char] = p.get(char, 0) + 1
        
    for char in p:
        p[char] /= len(text)
    
    return -sum(p_i * math.log2(p_i) for p_i in p.values())

In [8]:
def get_key(ciphertext):
    most_freq = ["ст", "но", "то", "на", "ен"]
    bi_freq = list(get_freqs(ciphertext).keys())[:5]

    for X1, X2 in permutations(most_freq, 2):
        X1n = bigram_to_int(X1)
        X2n = bigram_to_int(X2)
        for Y1, Y2 in permutations(bi_freq, 2):
            Y1n = bigram_to_int(Y1)
            Y2n = bigram_to_int(Y2)

            A = (X1n - X2n) % mod
            B = (Y1n - Y2n) % mod

            result = solve_linear(A, B, mod)

            for a in result:
                if math.gcd(a, mod) != 1:
                    continue

                b = (Y1n - a * X1n) % mod

                pt = decrypt(ciphertext, a, b)
                if pt is None:
                    continue
                
                if (entropy(pt) < 4.6): # обчислюю ентропію, щоб визначити, чи є текст осмисленим
                    print(f"Ключ: a={a}, b={b}")
                    print(pt[:120])

get_key(text)

Ключ: a=99, b=60
раннеераннееутропервыеотсветызаринакрышезаокномвселистьянадеревьяхвздрагиваютотзываясьнамалейшеедуновениепредрассветного
Ключ: a=99, b=60
раннеераннееутропервыеотсветызаринакрышезаокномвселистьянадеревьяхвздрагиваютотзываясьнамалейшеедуновениепредрассветного


In [9]:
with open("data\\13_decoded.txt", 'w', encoding='utf-8') as file:
    file.write(decrypt(text, 99, 60))